## Задача

- сделать анализ классификации текста с помощью LLM Qwen2-7B-Instruct
- сформировать classification_report с метриками
- замерить время получения предсказаний LLM и логрег
- сравнить метрики LLM и логрег
- описать результаты и сделать выводы

https://huggingface.co/Qwen/Qwen2-7B-Instruct - карточка модели<br>
Тут описано, что она из себя представляет и как её использовать.

In [ ]:
# установить необходимые библиотеки

In [1]:
!pip install fuzzywuzzy datasets transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 23.7 MB/s eta 0:00:000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 154.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 106.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 32.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 140.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.1/436.1 kB 134.6 MB/s eta 0:00:00
     ━━━

In [40]:
from typing import Dict, Union, List
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import Pipeline

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Qwen2ForSequenceClassification
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [39]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore")


In [32]:
# функция для подбора промпта для llm
def prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    def create_prompt(text: str, categories: List[str]) -> Dict[str, str]:
        # формируем текстовый запрос
        categories_str = ', '.join(categories)
        prompt = (f"Классифицируй следующий текст по одной из следующих категорий: {categories_str},  исключая любые друие категории." f"Текст: {text}")
        return [{'role': 'system', 'content': 'ты являешься сетью для классификации текста и должен ответить одним словом - категорией текста, больше слов не надо'}, {"role": "user", "content": prompt}]

    # если много текстов, то генерируем список промтов для каждого
    if isinstance(text, list):
        messages = [create_prompt(t, categories) for t in text]
    else:
        messages = [create_prompt(text, categories)]

    return messages


In [6]:
# загрузка модели Qwen2
model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2-7B-Instruct',offload_buffers=True)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-7B-Instruct")

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.08s/it]


In [8]:
# загрузка датасета по выборкам train, validation, test
def load_sib200_ru():
    """
    Загружает SIB200 датасет с выборками train, validation и test.
    """
    dataset = load_dataset("Davlan/sib200","rus_Cyrl")
    train_dataset = dataset['train']
    validation_dataset = dataset['validation']
    test_dataset = dataset['test']
    return dataset, train_dataset, validation_dataset, test_dataset

dataset, train_dataset, validation_dataset, test_dataset = load_sib200_ru()

Generating train split: 701 examples [00:00, 46246.40 examples/s]
Generating validation split: 99 examples [00:00, 31497.85 examples/s]
Generating test split: 204 examples [00:00, 47238.89 examples/s]


In [9]:
# список всех категорий из всех выборок
print_categ = list(set(dataset["train"]["category"]))
print_categ

['politics',
 'health',
 'entertainment',
 'science/technology',
 'sports',
 'geography',
 'travel']

In [41]:
# добавить в список фичей колонку 'message_for_llm', которая получится в результате применения функции prepare_message_for_llm к текстам
def add_column(dataset_val):
  dataset_val['message_for_llm'] = prepare_message_for_llm(dataset_val['text'], print_categ)
  return dataset_val
test_dataset = test_dataset.map(add_column)
train_dataset = train_dataset.map(add_column)
validation_dataset = validation_dataset.map(add_column)
test_dataset[1]['message_for_llm']

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 8153.72 examples/s]


[[{'content': 'ты являешься сетью для классификации текста и должен ответить одним словом - категорией текста, больше слов не надо',
   'role': 'system'},
  {'content': 'Классифицируй следующий текст по одной из следующих категорий: politics, health, entertainment, science/technology, sports, geography, travel,  исключая любые друие категории.Текст: Атомная бомба работает на том принципе, что для того, чтобы много протонов и нейтронов находились в одном ядре, необходима энергия.',
   'role': 'user'}]]

In [33]:
# получить предсказания для валидационной выборки и сформировать classification_report
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
responses = []

def predict(dataset):
    # Проходим по каждому элементу в датасете
    for entry in dataset:
        start_time = time.time()

        # ролучаем сообщение и применяем шаблон
        message = entry['message_for_llm']
        template_applied = tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=True
        )

        # токенизируем сообщение и отправляем на устройство
        inputs = tokenizer(template_applied, return_tensors='pt').to(device)


        generated_output = model.generate(
            inputs.input_ids,
            max_new_tokens=25
        )

        # отсекаем входные токены и оставляем только новые
        truncated_output = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(inputs.input_ids, generated_output)
        ]

        # декодируем и добавляем результат в список
        decoded_response = tokenizer.batch_decode(truncated_output, skip_special_tokens=True)[0]
        responses.append(decoded_response)

        # выводим результат и время выполнения для текущего сообщения
        elapsed_time = time.time() - start_time
        print(f"{decoded_response}, time = {elapsed_time:.2f} seconds")

total_start_time = time.time()
predict(validation_dataset)
total_elapsed_time = time.time() - total_start_time
print(f"Total time: {total_elapsed_time:.2f} seconds")


sports, time = 0.21 seconds
travel, time = 0.19 seconds
geography, time = 0.23 seconds
science/technology, time = 0.26 seconds
geography, time = 0.33 seconds
sports, time = 0.18 seconds
sports, time = 0.29 seconds
geography, time = 0.23 seconds
science/technology, time = 0.27 seconds
science/technology, time = 0.25 seconds
politics, time = 0.23 seconds
science/technology, time = 0.38 seconds
history, time = 0.26 seconds
travel, time = 0.29 seconds
transport, time = 0.29 seconds
science/technology, time = 0.26 seconds
health, time = 0.19 seconds
science/technology, time = 0.26 seconds
entertainment, time = 0.23 seconds
health, time = 0.19 seconds
politics, time = 0.23 seconds
geography, time = 0.22 seconds
sports, time = 0.29 seconds
science/technology, time = 0.26 seconds
geography, time = 0.23 seconds
travel, time = 0.19 seconds
science/technology, time = 0.37 seconds
health, time = 0.18 seconds
science/technology, time = 0.26 seconds
sports, time = 0.29 seconds
geography, time = 0.33

In [34]:
report = classification_report(validation_dataset['category'], responses)
print(report)


                    precision    recall  f1-score   support

           casinos       0.00      0.00      0.00         0
           culture       0.00      0.00      0.00         0
     entertainment       0.75      0.33      0.46         9
         geography       0.50      0.75      0.60         8
            health       0.88      0.64      0.74        11
           history       0.00      0.00      0.00         0
          politics       0.83      0.71      0.77        14
          religion       0.00      0.00      0.00         0
science/technology       0.77      0.92      0.84        25
            sports       0.92      0.92      0.92        12
        technology       0.00      0.00      0.00         0
           traffic       0.00      0.00      0.00         0
         transport       0.00      0.00      0.00         0
            travel       0.75      0.45      0.56        20

          accuracy                           0.70        99
         macro avg       0.39      0.3

In [35]:
# сделать то же самое для тестовой выборки
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
responses_test = []

def predict_test(dataset):
    for entry in dataset:
        start_time = time.time()

        # получаем сообщение и применяем шаблон
        message = entry['message_for_llm']
        template_applied = tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=True
        )

        # токенизируем сообщение и отправляем на устройство
        inputs = tokenizer(template_applied, return_tensors='pt').to(device)


        generated_output = model.generate(
            inputs.input_ids,
            max_new_tokens=35
        )

        # отсекаем входные токены и оставляем только новые
        truncated_output = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(inputs.input_ids, generated_output)
        ]

        # декодируем и добавляем результат в список
        decoded_response = tokenizer.batch_decode(truncated_output, skip_special_tokens=True)[0]
        responses_test.append(decoded_response)

        # Выводим результат и время выполнения для текущего сообщения
        elapsed_time = time.time() - start_time
        print(f"{decoded_response}, time = {elapsed_time:.2f} seconds")


total_start_time = time.time()
predict_test(test_dataset)
total_elapsed_time = time.time() - total_start_time
print(f"Total time: {total_elapsed_time:.2f} seconds")

science/technology, time = 0.29 seconds
science/technology, time = 0.26 seconds
science/technology, time = 0.26 seconds
science/technology, time = 0.27 seconds
science/technology, time = 0.27 seconds
science/technology, time = 0.27 seconds
science/technology, time = 0.26 seconds
biology, time = 0.19 seconds
science/technology, time = 0.37 seconds
science/technology, time = 0.27 seconds
science/technology, time = 0.27 seconds
science/technology, time = 0.37 seconds
science/technology, time = 0.37 seconds
science/technology, time = 0.37 seconds
science/technology, time = 0.26 seconds
science/technology, time = 0.27 seconds
science/technology, time = 0.26 seconds
science/technology, time = 0.37 seconds
science/technology, time = 0.26 seconds
science/technology, time = 0.27 seconds
science/technology, time = 0.26 seconds
science/technology, time = 0.37 seconds
science/technology, time = 0.26 seconds
science/technology, time = 0.37 seconds
science/technology, time = 0.37 seconds
science/tec

In [26]:
test_dataset['category']

['science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/technology',
 'science/t

In [36]:
report = classification_report(test_dataset['category'],responses_test)
print(report)

                    precision    recall  f1-score   support

     Entertainment       0.00      0.00      0.00         0
          Politics       0.00      0.00      0.00         0
         astronomy       0.00      0.00      0.00         0
           biology       0.00      0.00      0.00         0
         education       0.00      0.00      0.00         0
     entertainment       0.86      0.32      0.46        19
         geography       0.59      0.76      0.67        17
            health       0.89      0.73      0.80        22
           history       0.00      0.00      0.00         0
        literature       0.00      0.00      0.00         0
         migration       0.00      0.00      0.00         0
          politics       0.93      0.83      0.88        30
science/technology       0.75      0.96      0.84        51
            sports       0.95      0.80      0.87        25
         transport       0.00      0.00      0.00         0
    transportation       0.00      0.00

In [ ]:
# замерить время получения одного предсказания и предсказаний по всему датасету, сравнить с временем получения предсказаний с помощью логрег

In [13]:
# создаем pipeline с использованием TfidfVectorizer и LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000000)),  # Ограничиваем количество признаков
    ('logreg', LogisticRegression(max_iter=10000, random_state=42))
])

In [14]:
# обучаем логистическую регрессию на обучающей выборке
start_time = time.time()
pipeline.fit(train_dataset['text'], train_dataset['category'])
train_elapsed_time = time.time() - start_time
print(f"Training time for Logistic Regression: {train_elapsed_time:.2f} seconds")


Training time for Logistic Regression: 0.39 seconds


In [15]:
# предсказания на валидационной выборке
start_time = time.time()
validation_predictions = pipeline.predict(validation_dataset['text'])
validation_elapsed_time = time.time() - start_time
print(f"Prediction time for validation set with Logistic Regression: {validation_elapsed_time:.2f} seconds")


Prediction time for validation set with Logistic Regression: 0.01 seconds


In [16]:
# создаем отчет по метрикам для валидационной выборки
validation_report = classification_report(validation_dataset['category'], validation_predictions)
print("Validation set classification report (Logistic Regression):")
print(validation_report)


Validation set classification report (Logistic Regression):
                    precision    recall  f1-score   support

     entertainment       0.00      0.00      0.00         9
         geography       0.00      0.00      0.00         8
            health       0.00      0.00      0.00        11
          politics       0.67      0.43      0.52        14
science/technology       0.35      0.92      0.51        25
            sports       0.80      0.33      0.47        12
            travel       0.50      0.50      0.50        20

          accuracy                           0.43        99
         macro avg       0.33      0.31      0.29        99
      weighted avg       0.38      0.43      0.36        99



In [17]:
# предсказания на тестовой выборке
start_time = time.time()
test_predictions = pipeline.predict(test_dataset['text'])
test_elapsed_time = time.time() - start_time
print(f"Prediction time for test set with Logistic Regression: {test_elapsed_time:.2f} seconds")


Prediction time for test set with Logistic Regression: 0.01 seconds


In [18]:
# создаем отчет по метрикам для тестовой выборки
test_report = classification_report(test_dataset['category'], test_predictions)
print("Test set classification report (Logistic Regression):")
print(test_report)


Test set classification report (Logistic Regression):
                    precision    recall  f1-score   support

     entertainment       0.00      0.00      0.00        19
         geography       1.00      0.06      0.11        17
            health       0.00      0.00      0.00        22
          politics       0.82      0.47      0.60        30
science/technology       0.35      0.86      0.50        51
            sports       1.00      0.24      0.39        25
            travel       0.50      0.70      0.58        40

          accuracy                           0.46       204
         macro avg       0.53      0.33      0.31       204
      weighted avg       0.51      0.46      0.38       204



In [ ]:
# сделать вывод о полученных результатах

Ну метрики получились примертно одинаковые, но логрег обучилсь почти многвенно что нормально (наверное), с Qwen вообще проблем было много, напчиная с написание кода.
я ставил разное ограничение длины нового токена, на метрики не повлияло существенно


# update: 
попробовал модель Qwen2-7B-Instruct, стало явно лучше после Qwen2-0.5B-Instruct, что явно очевидно потому что кол-во параметров больше. результаты получились все ровно хуже чем на bert(там 0.91 accuracy на тесте). Пришлось поиграться с промтом, бывало что писала лишнюю инфу и ещё добавила категорию японскую какую-то, пришлось наиболее уточнять чтобы не было других категорий и писала одним словом а не фразой.